aggregateMessages其实和之前介绍过的[DGL的更新方式](https://github.com/zhulei227/GNN_Notes)类似，主要包括两部操作：   

1.利用边以及节点信息生成新的message    
2.目标节点聚合这些message  

所以，aggregateMessages函数的输入参数也是两个函数，第一个函数声明如何生产新message，第二个函数声明如何聚合这些message，下面用俩理智来直接实操一下

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@65af9124
sc = org.apache.spark.SparkContext@68639252


org.apache.spark.SparkContext@68639252

In [2]:
val verticesRDD=sc.parallelize(Array((0L,"a"),(1L,"b"),(2L,"c"),(3L,"d"),(4L,"e"),(5L,"f")))
val edgesRDD=sc.parallelize(Array(Edge(0L,1L,"r1"),Edge(0L,2L,"r1"),Edge(1L,3L,"r2"),Edge(2L,3L,"r3"),Edge(2L,4L,"r2"),Edge(5L,4L,"r3")))
val graph=Graph(verticesRDD,edgesRDD)

verticesRDD = ParallelCollectionRDD[0] at parallelize at <console>:31
edgesRDD = ParallelCollectionRDD[1] at parallelize at <console>:32
graph = org.apache.spark.graphx.impl.GraphImpl@393fb320


org.apache.spark.graphx.impl.GraphImpl@393fb320

### 一.求入度  
可以拆解为如下两步：   

1.源节点发送给目标节点信息：1  
2.目标节点求和所有的1  
![avatar](./pic/入度.png)

In [3]:
graph.aggregateMessages[Int]( //这里的Int指明Message的数据类型
    _.sendToDst(1),//第一个函数表示从src往dst发送1
    _+_  //第二个函数对这些1求和
).take(10)  //注意：返回数据类型是RDD

Array((4,2), (2,1), (1,1), (3,2))

### 二.求最远节点距离

求每个节点距离它的最远节点的距离，可以如下拆解：  
0：每个节点属性设置为attr=0   
1：源节点发送给目标节点信息为srcAttr+1   
2：目标节点对所有信息取max，并更新为当前节点的attr  

重复1～2直到收敛，流程如下，还以上图为例   

![avatar](./pic/最远距离.png)

In [4]:
var newGraph=graph.mapVertices((vid,vd)=>0)
for(i<-(1 to 10)){
    val vRDD=newGraph.aggregateMessages[Int](
                        (edge:EdgeContext[Int,String,Int])=>edge.sendToDst(edge.srcAttr+1),
                        (a:Int,b:Int)=>math.max(a,b)
                    )
    newGraph=Graph(vRDD,newGraph.edges)
}

newGraph = org.apache.spark.graphx.impl.GraphImpl@13d575ce


org.apache.spark.graphx.impl.GraphImpl@13d575ce

In [5]:
newGraph.vertices.take(10)

Array((4,2), (0,0), (2,1), (1,1), (3,2), (5,0))

aggregateMessages很大的方便了我们在图上的计算，但还些不足，比如：   

1：aggregateMessages每次返回的是一个RDD，我还需要手动将其封装为Graph  
2：aggregateMessages通常需要反复迭代多次才能得到最终结果，需要自己再外围定义for循环，有时还需要自己判断收敛条件   
3：aggregateMessages得到的是边上聚合结果，如果这个结果还需要与原始节点上的属性进行计算，这部分需要自己额外完成    

所以，为了进一步简化图上的计算，Pregel算子来了，它也是后续诸多算法常用的API